In [ ]:
import pandas as pd

df=pd.read_csv('CH3Thailand.csv')
df.head()

In [ ]:
!pip install pythainlp
!pip install marisa_trie

from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
import marisa_trie
from pythainlp.corpus import thai_stopwords

added_words = ['กรมสมเด็จพระเทพฯ','ศูนย์การเรียน','ตำรวจตระเวนชายแดน','วาร์ป','อันดับ','มิญ่า','คาลิฟา','วันพีซ','ซาร่า','ย้อนหลัง','น้ำมันมะพร้าว',
               'ณเดชน์','มงกุฎ','จรณ','อินเนอร์','แมนเนเจอร์','เกนหลง','ลูกยาง','พระพุทธรูป','อุลตร้าแมน','เมนต์','อังกอร์','ชาลิดา','เบลล่า','เบลล์','วีกิจ']

my_punc = set(['"','“','”','ๆ','!','-','\xa0•••',"‘",')','(','0','1','2','3','4','5','6','7','8','9','...','.','/',"’",':','%',
               "'",'#'])

all_words = set(thai_words())
for word in added_words:
  all_words.add(word)
c_dict = marisa_trie.Trie(all_words)

def extract_words(title,c_dict):
  words = ''
  for token in title.split():
    if token[0] == '#': # remove
      continue
    elif 'http'in token or 'www' in token:
      continue
    else:
      words += ' '+cut_words(token,c_dict)
  return words

def cut_words(title,c_dict):

  words = word_tokenize(title,engine='newmm',custom_dict=c_dict,keep_whitespace=False)
  words = [word for word in words if word not in my_punc and word not in thai_stopwords()]
  words = [word for word in words if len(word) > 1]
  return ' '.join(words)

In [ ]:
import pandas as pd

files = ['CH3Thailand.csv','ejan2016.csv','punpromotion.csv']
out_file = pd.DataFrame(columns=['message','class'])
for file in files:
  df = pd.read_csv(file)
  for msg in df.message:
    out_file=out_file.append({'message':extract_words(msg,c_dict),'class':file},ignore_index=True)
out_file

,message,class
0,อย่า โกรธ แม่ ขาย 😭😭 ดู ย้อนหลัง ที่นี่,CH3Thailand.csv
1,แอคชั่น เต็ม แถม ซึ้ง ฉาก หวาน ดู เบื้องหลัง ...,CH3Thailand.csv
2,ร้องไห้ ไหว ทด กลายเป็น ทาส เด็ก ดู ย้อนหลัง ...,CH3Thailand.csv
3,รื้อ กระเป๋า เผ่า ส่อง ไอเทม เด็ด ดู ติดตาม ...,CH3Thailand.csv
4,มีความผิด ที่จะ โดน เฆี่ยน ทด ใจเด็ด ดู ย้อน...,CH3Thailand.csv
...,...,...
4550,คุ้ม คุ้ม คุ้ม แบ่งปัน ส่วนลด โปรโมชั่น บัตรเ...,punpromotion.csv
4551,Mo-Mo Paradise กิน อั้น ชาบู ชาบู สุกี้ สไตล์...,punpromotion.csv
4552,บัตรเครดิต ส่วนลด ไม่ต้อง เสียใจ ปัน โปร >> ...,punpromotion.csv
4553,เบื่อ ไหม พลาด ส่วนลด บัตรเครดิต ปัน โปร,punpromotion.csv


In [ ]:
out_file.to_csv('text_data.csv')